In [ ]:
import torch, torchtext, numpy as np
import pandas as pd, csv
from torch import nn, optim
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import pdb
from keras.models import Sequential
from tensorflow import keras
from scipy import signal
import os

In [ ]:
# assign directory
directory = "drive/MyDrive/MINT_DATA"
 
# iterate over files in
# that directory
dataset=[]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        ds = pd.read_csv("drive/MyDrive/MINT_DATA/OpenBCI-RAW-2022-10-21_18-48-52.csv")
        ds = ds.tail(-123)
        dataset.append(ds)

In [ ]:
dataset[0].shape

(5029, 25)

In [ ]:
for data in dataset:
  data.drop(data.iloc[:, 9:25], axis=1, inplace=True)

In [ ]:
dataset[0].shape

(5029, 9)

In [ ]:
i =0;
for data in dataset:
  if data.shape[0] < 4609:
    i=1
i

0

In [ ]:
data_train=dataset[0:40]
data_test=dataset[40:52]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler =MinMaxScaler()
i=0;
for data in dataset[0:40]:
  data_train[i] = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)
  i=i+1
i=0
for data in dataset[40:52]:
  data_test[i] = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)
  i=i+1

In [ ]:
import tqdm #0 is green, 1 is blue, 2 is red
n=256*6
feature_names=list(data_train[0].drop('Sample Index',axis=1).columns)
X=[]
Y=[]
indexes=[]
for data in data_train:
  norm_data_x =data[feature_names]
  for i in tqdm.tqdm_notebook(range(3)):
    X.append(norm_data_x.iloc[i*n:i*n+n].  values)
    Y.append(i)
    indexes.append(data.index[i+n-1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
print(X[1])
print(Y[1])

[[0.56376588 0.54005228 0.71504798 ... 0.37288611 0.31276469 0.25396927]
 [0.57009795 0.54306599 0.71527421 ... 0.37219102 0.31381055 0.26358159]
 [0.57607666 0.54812767 0.71538733 ... 0.36990282 0.31017581 0.2660843 ]
 ...
 [0.35325296 0.3482409  0.42475459 ... 0.19926778 0.61389967 0.5258879 ]
 [0.3540586  0.34952297 0.42503325 ... 0.19877991 0.61423479 0.53000064]
 [0.3534791  0.35018898 0.42501945 ... 0.1966385  0.61112666 0.52995282]]
1


In [ ]:
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.optimizers import adam_v2
n_steps=256*6
n_features=8
model=Sequential()
model.add(LSTM(100, activation='relu', return_sequences=False, input_shape=(n_steps, n_features)))
model.add(Dense(1))
opt = adam_v2.Adam(lr=0.0005, decay=1e-6)

model.compile(optimizer=opt, loss='mse', metrics=['mse','mape'])

In [ ]:
history=model.fit(np.asarray(X),np.asarray(Y),batch_size=100,epochs=15)

Epoch 1/15
2/2 [==============================] - 6s 876ms/step - loss: 1.3760 - mse: 1.3760 - mape: 9835420.0000
Epoch 2/15
2/2 [==============================] - 4s 846ms/step - loss: 1.2056 - mse: 1.2056 - mape: 16848068.0000
Epoch 3/15
2/2 [==============================] - 4s 849ms/step - loss: 1.0426 - mse: 1.0426 - mape: 39217080.0000
Epoch 4/15
2/2 [==============================] - 4s 865ms/step - loss: 0.8882 - mse: 0.8882 - mape: 60893916.0000
Epoch 5/15
2/2 [==============================] - 4s 839ms/step - loss: 0.7332 - mse: 0.7332 - mape: 82588720.0000
Epoch 6/15
2/2 [==============================] - 4s 806ms/step - loss: 0.5759 - mse: 0.5759 - mape: 108859680.0000
Epoch 7/15
2/2 [==============================] - 4s 883ms/step - loss: 0.4051 - mse: 0.4051 - mape: 135164384.0000
Epoch 8/15
2/2 [==============================] - 4s 878ms/step - loss: 0.2396 - mse: 0.2396 - mape: 168463264.0000
Epoch 9/15
2/2 [==============================] - 4s 861ms/step - loss: 0.1729

In [ ]:
n=256*6
feature_names=list(data_test[0].drop('Sample Index',axis=1).columns)
X_test=[]
Y_test=[]
indexes=[]
for data in data_test:
  norm_data_x =data[feature_names]
  for i in tqdm.tqdm_notebook(range(3)):
    X_test.append(norm_data_x.iloc[i*n:i*n+n].  values)
    Y_test.append(i)
    indexes.append(data.index[i+n-1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
predictions = model.predict(np.asarray(X_test))
predictions = pd.DataFrame(predictions).rename(columns={0: 'prediction'})
predictions

2/2 [==============================] - 1s 126ms/step


,prediction
0,0.635719
1,0.991155
2,1.472177
3,0.635719
4,0.991155
5,1.472177
6,0.635719
7,0.991155
8,1.472177
9,0.635719


In [ ]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))